In [ ]:
from google.colab import drive
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from nltk.tokenize import word_tokenize
import nltk
from sklearn.metrics import accuracy_score
nltk.download('punkt')
import codecs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.71.72.106:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 6145056503130899981),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, 2583947746098275243),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, -3651025924268971260),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, -6469696853502400923),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3514717218816381940),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, -219972069111696133),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, -1250983248452068403),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1816859983468790401),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.71.72.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.71.72.106:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2021-04-28 06:48:41--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  15.0MB/s    in 44s     

2021-04-28 06:49:25 (14.9 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [ ]:
!unzip wiki-news-300d-1M.vec.zip

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [ ]:
class word_emb_model:
  def __init__(self,path):
      df = pd.read_csv(path)
      self.encoder = LabelEncoder()
      self.models_fasttext = {}
      self.history_fasttext = {}
      # self.accuracy_glove ={}
      df['text'] = df['text'].apply(lambda x: str(x))
      self.dfs_train = df[df['kfold']!=0]
      self.dfs_valid = df[df['kfold']==0]
      self.read_fasttext()
      self.data_per_fold_fasttext()

  def read_glove(self):
      embeddings_index = {}
      f = open('glove.twitter.27B.200d.txt')
      for line in f:
          values = line.split()
          word = values[0]
          coefs = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = coefs
      f.close()

      print('Found %s word vectors.' % len(embeddings_index))
      self.glove_embeddings_index = embeddings_index

  def read_fasttext(self):
      embeddings_index = {}
      f = codecs.open('wiki-news-300d-1M.vec', encoding='utf-8')
      for line in f:
          values = line.rstrip().rsplit(' ')
          word = values[0]
          coefs = np.asarray(values[1:], dtype='float32')
          embeddings_index[word] = coefs
      f.close()
      print('found %s word vectors' % len(embeddings_index))
      self.fasttext_embeddings_index = embeddings_index

  # def create_glove_embedding_matrix(self,embed_dim):
      # train = self.dfs_train
      # test = self.dfs_valid
      # x_train = train.text.values
      # y_train = train.label.values
      # x_test = test.text.values
      # y_test = test.label.values
      # word_tokenizer = Tokenizer()
      # word_tokenizer.fit_on_texts(x_train)
      # vocab_length = len(word_tokenizer.word_index) + 1
      # x_train_emb = word_tokenizer.texts_to_sequences(x_train)
      # x_test_emb = word_tokenizer.texts_to_sequences(x_test)
      
      # word_count = lambda sentence: len(word_tokenize(sentence))
      # longest_sentence = max(np.concatenate((x_train,x_test),axis = 0), key=word_count)
      # length_long_sentence = len(word_tokenize(longest_sentence))

      # x_train_padded = pad_sequences(x_train_emb, length_long_sentence, padding='post',truncating = 'post')
      # x_test_padded = pad_sequences(x_test_emb, length_long_sentence, padding='post',truncating = 'post')

      # encoded_Y_tr = self.encoder.fit_transform(y_train)
      # encoded_Y_ts = self.encoder.fit_transform(y_test)
      # dummy_y_tr = np_utils.to_categorical(encoded_Y_tr)
      # dummy_y_ts = np_utils.to_categorical(encoded_Y_ts)

      # embedding_matrix = np.zeros((vocab_length, embed_dim))
      # for word, index in word_tokenizer.word_index.items():
      #     embedding_vector = self.glove_embeddings_index.get(word)
      #     if embedding_vector is not None:
      #         embedding_matrix[index] = embedding_vector

      # # print(x_train_padded)
      # # print(x_test_padded)
      # return ((tf.convert_to_tensor(x_train_padded,np.int32),tf.convert_to_tensor(dummy_y_tr,np.float32)),(tf.convert_to_tensor(x_test_padded,np.int32),tf.convert_to_tensor(dummy_y_ts,np.float32)),(vocab_length,embedding_matrix,embed_dim,length_long_sentence))

  def create_fasttext_embedding_matrix(self,embed_dim):
      train = self.dfs_train
      test = self.dfs_valid
      x_train = train.text.values
      y_train = train.label.values
      x_test = test.text.values
      y_test = test.label.values
      word_tokenizer = Tokenizer()
      word_tokenizer.fit_on_texts(x_train)
      vocab_length = len(word_tokenizer.word_index) + 1
      x_train_emb = word_tokenizer.texts_to_sequences(x_train)
      x_test_emb = word_tokenizer.texts_to_sequences(x_test)
      
      word_count = lambda sentence: len(word_tokenize(sentence))
      longest_sentence = max(np.concatenate((x_train,x_test),axis = 0), key=word_count)
      length_long_sentence = len(word_tokenize(longest_sentence))

      x_train_padded = pad_sequences(x_train_emb, length_long_sentence, padding='post',truncating = 'post')
      x_test_padded = pad_sequences(x_test_emb, length_long_sentence, padding='post',truncating = 'post')

      encoded_Y_tr = self.encoder.fit_transform(y_train)
      encoded_Y_ts = self.encoder.fit_transform(y_test)
      dummy_y_tr = np_utils.to_categorical(encoded_Y_tr)
      dummy_y_ts = np_utils.to_categorical(encoded_Y_ts)

      embedding_matrix = np.zeros((vocab_length, embed_dim))
      for word, index in word_tokenizer.word_index.items():
          embedding_vector = self.fasttext_embeddings_index.get(word)
          if embedding_vector is not None:
              embedding_matrix[index] = embedding_vector

      # print(x_train_padded)
      # print(x_test_padded)
      return ((tf.convert_to_tensor(x_train_padded,np.int32),tf.convert_to_tensor(dummy_y_tr,np.float32)),(tf.convert_to_tensor(x_test_padded,np.int32),tf.convert_to_tensor(dummy_y_ts,np.float32)),(vocab_length,embedding_matrix,embed_dim,length_long_sentence))


  def data_per_fold_glove(self):
      # self.data_glove = {}
      embed_dim = 200
      self.data_glove = self.create_glove_embedding_matrix(embed_dim)

  def data_per_fold_fasttext(self):
      # self.data_fasttext = {}
      embed_dim = 300
      self.data_fasttext = self.create_fasttext_embedding_matrix(embed_dim)


  def process_fasttext(self):
      models = [self.CONV1D,self.SEP_CONV1D,self.CONV1D_BiLSTM,self.CONV1D_BiRNN,self.CONV1D_BiGRU,self.SEP_CONV1D_BiLSTM]
      for mod in models:
          name = str(mod)
          name = name.replace('self.','')
          m,h = self.run_training(mod)
          self.models_fasttext[name] = m
          self.history_fasttext[name] = h
          # self.accuracy_glove['Test accuracy score for '+name] = acc
    
  def get_models_fasttext(self):
    return self.models_fasttext

  def get_history_fasttext(self):
    return self.history_fasttext


  def run_training(self,model):
      mod = []
      his = []
      x,h= model()
      mod.append(x)
      his.append(h)
    
      
      return mod,his

  def SEP_CONV1D(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Dropout(0.5),
              tf.keras.layers.GlobalMaxPooling1D(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n SEPERABLE CONV1D \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his





  def CONV1D(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n CONV1D \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his#,accuracy_score(y_ts,y_pred)

  def BiLSTM(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(2, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n BiLSTM \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)

  def SEP_CONV1D_BiLSTM(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n SEPARABLE CONV1D BiLSTM \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)


  def CONV1D_BiLSTM(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n CONV1D BiLSTM \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)


  def BiRNN(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(2, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n BiRNN \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)

  def SEP_CONV1D_BiRNN(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(2, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n SEPARABLE CONV1D BiRNN \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)

  
  def CONV1D_BiRNN(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n CONV1D BiRNN \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)

  def BiGRU(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              # tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(2, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n BiGRU \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)
      
  def CONV1D_BiGRU(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.Conv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(5, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n CONV1D BiGRU \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)
      
  def SEP_CONV1D_BiGRU(self):
      data = self.data_fasttext
      x_tr,y_tr = data[0]
      x_ts,y_ts = data[1]
      vl,mat,dim,len_sen = data[2]
      
      with strategy.scope():
          model = tf.keras.models.Sequential([
              tf.keras.layers.Embedding(vl,dim,weights=[mat],input_length=len_sen,trainable=False),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              tf.keras.layers.SeparableConv1D(32,3,activation='relu'),
              tf.keras.layers.MaxPool1D(),
              # tf.keras.layers.Dropout(0.5),
              tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True, dropout=0.2, recurrent_dropout=0.3)),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(512, activation='sigmoid'),
              tf.keras.layers.Dense(256, activation='sigmoid'),
              tf.keras.layers.Dense(128, activation='sigmoid'),
              tf.keras.layers.Dense(64, activation='sigmoid'),
              tf.keras.layers.Dense(2, activation='softmax')])

          model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adam(),
          metrics=['accuracy'])
      x = model.to_json()
      print("\n SEPARABLE CONV1D BiGRU \n")
      model.summary()
      his = model.fit(x = x_tr,y=y_tr,validation_data=(x_ts,y_ts),epochs=100,callbacks=[EarlyStopping()])
      # y_pred = model.predict(x_ts)
      return x,his #,accuracy_score(y_ts,y_pred)
      
    

In [ ]:
# wem = word_emb_model('/content/drive/MyDrive/sarcasm data/train_sarc_final_with_folds.csv')
wem = word_emb_model('/content/drive/MyDrive/sentiment data/tr_sen_fin.csv')

found 999995 word vectors


In [ ]:
wem.process_fasttext()


 CONV1D 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 163, 300)          41134800  
_________________________________________________________________
conv1d (Conv1D)              (None, 161, 32)           28832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 80, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 78, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 39, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 37, 32)            3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 18, 32)  

In [ ]:
y = wem.get_history_fasttext()

In [ ]:
dfs = []
for k in y.keys():
  z = y[k][0].history
  df = pd.DataFrame.from_dict(z)
  df['epoch'] = [i+1 for i in range(len(df))]
  dfs.append(df)

In [ ]:
len(dfs)

6

In [ ]:
for k in y.keys():
  print(k)

<bound method word_emb_model.CONV1D of <__main__.word_emb_model object at 0x7f4d14e49b50>>
<bound method word_emb_model.SEP_CONV1D of <__main__.word_emb_model object at 0x7f4d14e49b50>>
<bound method word_emb_model.CONV1D_BiLSTM of <__main__.word_emb_model object at 0x7f4d14e49b50>>
<bound method word_emb_model.CONV1D_BiRNN of <__main__.word_emb_model object at 0x7f4d14e49b50>>
<bound method word_emb_model.CONV1D_BiGRU of <__main__.word_emb_model object at 0x7f4d14e49b50>>
<bound method word_emb_model.SEP_CONV1D_BiLSTM of <__main__.word_emb_model object at 0x7f4d14e49b50>>


In [ ]:
k = ['CONV1D','SEP_CONV1D','CONV1D_BiLSTM','CONV1D_BiRNN','CONV1D_BiGRU','SEP_CONV1D_BiLSTM']

In [ ]:
for i in range(len(dfs)):
  dfs[i].to_csv('/content/drive/MyDrive/sent_fasttext_300_'+k[i]+'.csv',index = False,header = True)